In [1]:
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
import os
from tensorflow.keras.layers import Conv2D ,MaxPooling2D,SimpleRNN,GRU,LSTM,Dense,Flatten,TimeDistributed

In [2]:
frame_w= 50
frame_h= 50

Dataset

In [3]:
path = "/content/drive/MyDrive/joon_del/"
dirs = os.listdir(path)
Dataset=[]
label=[]
for dir in dirs:
  subfolder = f"/content/drive/MyDrive/joon_del/{dir}"
  dataset = os.listdir(subfolder)
  for j in dataset:
    cap = cv2.VideoCapture(f'/content/drive/MyDrive/joon_del/{dir}/{j}')
    video=[]
    while(True):
      ret, frame = cap.read()
      if ret == False:
          break
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      img = cv2.resize(frame, dsize=(frame_w, frame_h))
      img = img / 255.
      video.append(img)
    Dataset.append(video)
    label.append(float(dir))

In [4]:
list = max(len(j) for j in Dataset)
for i, j in enumerate(Dataset):
  if len (j) < list:
    for j in range(list-len(j)):
      Dataset[i].append(np.zeros((frame_w, frame_h, 3)))

In [5]:
newdata = np.array(Dataset)
label = np.array(label)
label = label[..., np.newaxis]
X_train, X_test, y_train, y_test = train_test_split(newdata, label, test_size=0.2, random_state=42)

In [6]:
model_crnn = tf.keras.models.Sequential([
                  # CNN
                  TimeDistributed(Conv2D(64, (3, 3), activation="relu", input_shape=(None, frame_w,frame_h, 3))),
                  TimeDistributed(Conv2D(32, (3, 3), activation="relu", input_shape=(None, frame_w,frame_h, 3))),
                  TimeDistributed(Conv2D(16, (3, 3), activation="relu", input_shape=(None, frame_w,frame_h, 3))),
                  TimeDistributed(MaxPooling2D(pool_size=(2, 2))),

                  TimeDistributed(Flatten()),
                  # RNN
                  SimpleRNN(50, return_sequences=True),
                  SimpleRNN(20),
                  Dense(2, activation="softmax")
])

In [7]:
model_crnn.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

model_crnn.fit(X_train, y_train, batch_size=8, epochs=20)

Epoch 1/20
7/7 [==============================] - 30s 2s/step - loss: 0.7317 - accuracy: 0.5179
Epoch 2/20
7/7 [==============================] - 12s 2s/step - loss: 0.7406 - accuracy: 0.5357
Epoch 3/20
7/7 [==============================] - 13s 2s/step - loss: 0.6989 - accuracy: 0.4464
Epoch 4/20
7/7 [==============================] - 12s 2s/step - loss: 0.6828 - accuracy: 0.5000
Epoch 5/20
7/7 [==============================] - 12s 2s/step - loss: 0.6723 - accuracy: 0.5357
Epoch 6/20
7/7 [==============================] - 12s 2s/step - loss: 0.6748 - accuracy: 0.5536
Epoch 7/20
7/7 [==============================] - 12s 2s/step - loss: 0.6687 - accuracy: 0.5536
Epoch 8/20
7/7 [==============================] - 12s 2s/step - loss: 0.6793 - accuracy: 0.5000
Epoch 9/20
7/7 [==============================] - 12s 2s/step - loss: 0.6802 - accuracy: 0.5536
Epoch 10/20
7/7 [==============================] - 12s 2s/step - loss: 0.6884 - accuracy: 0.5179
Epoch 11/20
7/7 [======================

In [8]:
model_crnn.evaluate(X_test, y_test)

1/1 [==============================] - 7s 7s/step - loss: 0.7277 - accuracy: 0.2857


[0.7277035117149353, 0.2857142984867096]